# Python爬虫

---
## 1. 了解网页结构

首先要做的, 是使用 `Python` 来登录这个网页, 并打印出这个网页 `HTML` 的 source code. 注意, 因为网页中存在中文, 为了正常显示中文, `read()` 完以后, 我们要对读出来的文字进行转换, `decode()` 成可以正常显示中文的形式.

In [1]:
from urllib.request import urlopen

# if has Chinese, apply decode()
html = urlopen(
    "https://morvanzhou.github.io/static/scraping/basic-structure.html"
).read().decode('utf-8')
print(html)

<!DOCTYPE html>
<html lang="cn">
<head>
	<meta charset="UTF-8">
	<title>Scraping tutorial 1 | 莫烦Python</title>
	<link rel="icon" href="https://morvanzhou.github.io/static/img/description/tab_icon.png">
</head>
<body>
	<h1>爬虫测试1</h1>
	<p>
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
		<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	</p>

</body>
</html>


### 1.1 匹配网页内容

所以这里我们使用 `Python` 的正则表达式 `RegEx` 进行匹配文字, 筛选信息的工作. 我有一个很不错的[正则表达式的教程](https://morvanzhou.github.io/tutorials/python-basic/basic/13-10-regular-expression/), 如果是初级的网页匹配, 我们使用正则完全就可以了, 高级一点或者比较繁琐的匹配, 我还是推荐使用 [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

如果我们想用代码找到这个网页的 title, 我们就能这样写. 选好要使用的 tag 名称 `<title>`. 使用正则匹配.

In [2]:
import re
res = re.findall(r"<title>(.+?)</title>", html)
print("\nPage title is: ", res[0])


Page title is:  Scraping tutorial 1 | 莫烦Python


如果想要找到中间的那个段落 `<p>`, 我们使用下面方法, 因为这个段落在 HTML 中还夹杂着 tab, new line, 所以我们给一个 `flags=re.DOTALL` 来对这些 tab, new line 不敏感.

In [3]:
res = re.findall(r"<p>(.*?)</p>", html, flags=re.DOTALL)    # re.DOTALL if multi line
print("\nPage paragraph is: ", res[0])


Page paragraph is:  
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
		<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	


最后一个练习是找一找所有的链接, 这个比较有用, 有时候你想找到网页里的链接, 然后下载一些内容到电脑里, 就靠这样的途径了.

In [4]:
res = re.findall(r'href="(.*?)"', html)
print("\nAll links: ", res)


All links:  ['https://morvanzhou.github.io/static/img/description/tab_icon.png', 'https://morvanzhou.github.io/', 'https://morvanzhou.github.io/tutorials/data-manipulation/scraping/']


---
## 2. BeartifulSoup 解析网页：基础

学习资料：
* BeautifulSoup [中文官网](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)

我们总结一下爬网页的流程, 让你对 BeautifulSoup 有一个更好的定位.

1. 选择要爬的网址 (url)
2. 使用 python 登录上这个网址 (urlopen等)
3. 读取网页信息 (read() 出来)
4. **将读取的信息放入 BeautifulSoup**
5. **使用 BeautifulSoup 选取 tag 信息等 (代替正则表达式)**

### 2.1 安装

    # Python 3+
    pip3 install beautifulsoup4
    
注意在名字后面还有个 “4”, 可能是代表第4版吧. 如果你在安装的时候遇到任何问题, 请参考他们[官网](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/#id5)上的解决方案.

### 2.2 简单实用方法

BeautifulSoup 使用起来非常简单, 我们先按常规读取网页.

In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

# if has Chinese, apply decode()
html = urlopen(
    "https://morvanzhou.github.io/static/scraping/basic-structure.html"
).read().decode('utf-8')
print(html)

<!DOCTYPE html>
<html lang="cn">
<head>
	<meta charset="UTF-8">
	<title>Scraping tutorial 1 | 莫烦Python</title>
	<link rel="icon" href="https://morvanzhou.github.io/static/img/description/tab_icon.png">
</head>
<body>
	<h1>爬虫测试1</h1>
	<p>
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
		<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	</p>

</body>
</html>


回顾一下, 每张网页中, 都有两大块, 一个是 `<head>`, 一个是 `<body>`, 我们等会用 `BeautifulSoup` 来找到 body 中的段落 `<p>` 和所有链接 `<a>`.

读取这个网页信息, 我们将要加载进 BeautifulSoup, 以 `lxml` 的这种形式加载. 除了 `lxml`, 其实还有[很多形式的解析器](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/#id9), 不过大家都推荐使用 `lxml` 的形式. 然后 `soup` 里面就有着这个 HTML 的所有信息. 如果你要输出 `<h1>` 标题, 可以就直接 `soup.h1`.

In [6]:
import lxml

soup = BeautifulSoup(html, features='lxml')
print(soup.h1)

print('\n', soup.p)

<h1>爬虫测试1</h1>

 <p>
		这是一个在 <a href="https://morvanzhou.github.io/">莫烦Python</a>
<a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/">爬虫教程</a> 中的简单测试.
	</p>


如果网页中有过个同样的 tag, 比如链接 `<a>`, 我们可以使用 `find_all()` 来找到所有的选项. 因为我们真正的 link 不是在 `<a>` 中间 `</a>`, 而是在 `<a href="link">` 里面, 也可以看做是 `<a>` 的一个属性. 我们能用像 Python 字典的形式, 用 key 来读取 `l["href"]`.

In [7]:
all_href = soup.find_all('a')
all_href = [l['href'] for l in all_href]
print('\n', all_href)


 ['https://morvanzhou.github.io/', 'https://morvanzhou.github.io/tutorials/data-manipulation/scraping/']


懂得这些还是远远不够的, 真实情况往往比这些复杂. BeautifulSoup 还有很多其他的选择”增强器”. 下次, 我们来了解一些 CSS 的概念, 用 BeautifulSoup 加上 CSS 来选择内容.

---
## 3. BeautifulSoup 解析网页：CSS

In [8]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

# if has Chinese, apply decode()
html = urlopen(
    "https://morvanzhou.github.io/static/scraping/list.html"
).read().decode('utf-8')
print(html)

<!DOCTYPE html>
<html lang="cn">
<head>
	<meta charset="UTF-8">
	<title>爬虫练习 列表 class | 莫烦 Python</title>
	<style>
	.jan {
		background-color: yellow;
	}
	.feb {
		font-size: 25px;
	}
	.month {
		color: red;
	}
	</style>
</head>

<body>

<h1>列表 爬虫练习</h1>

<p>这是一个在 <a href="https://morvanzhou.github.io/" >莫烦 Python</a> 的 <a href="https://morvanzhou.github.io/tutorials/data-manipulation/scraping/" >爬虫教程</a>
	里无敌简单的网页, 所有的 code 让你一目了然, 清晰无比.</p>

<ul>
	<li class="month">一月</li>
	<ul class="jan">
		<li>一月一号</li>
		<li>一月二号</li>
		<li>一月三号</li>
	</ul>
	<li class="feb month">二月</li>
	<li class="month">三月</li>
	<li class="month">四月</li>
	<li class="month">五月</li>
</ul>

</body>
</html>


在 `<head>` 中, 你会发现有这样一些东西被放在 `<style>` 里面, 这些东西都是某些 class 的 CSS 代码. 比如 `jan` 就是一个 class. `jan` 这个类掌控了这个类型的背景颜色. 所以在 `<ul class="jan">` 这里, 这个 ul 的背景颜色就是黄色的. 而如果是 `month` 这个类, 它们的字体颜色就是红色.

这样, 我们就知道, 有时候, 网页中, 这种 class 归类一些组件还是很有用的. 比如我就想找 `jan` 下面的这些 `<li>`. 我就能通过寻找 `class="jan"` 找到它们. BeautifulSoup 就能这么干.

### 3.1 按 Class 匹配

按 Class 匹配很简单. 比如我要找所有 class=month 的信息. 并打印出它们的 tag 内文字.

In [9]:
soup = BeautifulSoup(html, features='lxml')

# use class to narrow search
month = soup.find_all('li', {"class": "month"})
for m in month:
    print(m.get_text())

一月
二月
三月
四月
五月


或者找到 class=jan 的信息. 然后在 `<ul>` 下面继续找 `<ul>` 内部的 `<li>` 信息. 这样一层层嵌套的信息, 非常容易找到.

In [10]:
jan = soup.find('ul', {"class": 'jan'})
d_jan = jan.find_all('li')              # use jan as a parent
for d in d_jan:
    print(d.get_text())

一月一号
一月二号
一月三号


如果想要找到一些有着一定格式的信息, 比如使用正则表达来寻找相类似的信息, 我们在 BeautifulSoup 中也能嵌入正则表达式, 让 BeautifulSoup 更为强大.

---
## 4. BeautifulSoup 解析网页：正则表达

### 4.1 正则匹配

导入正则模块 re.

In [11]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

# if has Chinese, apply decode()
html = urlopen(
    "https://morvanzhou.github.io/static/scraping/table.html"
).read().decode('utf-8')

我们可以用 `soup` 将这些 `<img>` tag 全部找出来, 但是每一个 img 的链接(src)都可能不同. 或者每一个图片有的可能是 jpg 有的是 png, 如果我们只想挑选 jpg 形式的图片, 我们就可以用这样一个正则 `r'.*?\.jpg'` 来选取. 把正则的 compile 形式放到 BeautifulSoup 的功能中, 就能选到符合要求的图片链接了.

In [12]:
import lxml

soup = BeautifulSoup(html, features='lxml')

img_links = soup.find_all("img", {"src": re.compile('.*?\.jpg')})
for link in img_links:
    print(link['src'])

https://morvanzhou.github.io/static/img/course_cover/tf.jpg
https://morvanzhou.github.io/static/img/course_cover/rl.jpg
https://morvanzhou.github.io/static/img/course_cover/scraping.jpg


又或者我们发现, 我想选一些课程的链接, 而这些链接都有统一的形式, 就是开头都会有 `https://morvan.`, 那我就将这个定为一个正则的规则, 让 BeautifulSoup 帮我找到符合这个规则的链接.

In [13]:
course_links = soup.find_all('a', {'href': re.compile('https://morvan.*')})
for link in course_links:
    print(link['href'])

https://morvanzhou.github.io/
https://morvanzhou.github.io/tutorials/data-manipulation/scraping/
https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/
https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/
https://morvanzhou.github.io/tutorials/data-manipulation/scraping/


---
## 5. 小练习：爬百度百科

### 5.1 观看规律

这个爬虫说实在的, 并不难, 只有20+行代码. 但是却能让它游走在百度百科的知识的海洋中. 首先我们需要定义一个起始网页, 我选择了 “[网页爬虫](https://baike.baidu.com/item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB)”. 我们发现, 页面中有一些链接, 指向百度百科中的另外一些词条, 比如说下面这样.

    <a target="_blank" href="/item/%E8%9C%98%E8%9B%9B/8135707" data-lemmaid="8135707">蜘蛛</a>
    <a target="_blank" href="/item/%E8%A0%95%E8%99%AB">蠕虫</a>
    <a target="_blank" href="/item/%E9%80%9A%E7%94%A8%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E">通用搜索引擎</a>
    
通过观察, 我们发现, 链接有些共通之处. 它们都是 `/item/` 开头, 夹杂着一些 `%E9` 这样的东西. 但是仔细搜索一下, 发现还有一些以 `/item/` 开头的, 却不是词条. 比如

    <a href="/item/史记·2016?fr=navbar" target="_blank">史记·2016</a>
    
我想, 我们需要对这些链接做一些筛选, 之前提到 的用 BeautifulSoup 和 正则表达式来筛选应该用得上. 有了些思路, 我们开始写代码吧.

### 5.2 制作爬虫

导入一些模块, 设置起始页. 并将 `/item/...` 的网页都放在 `his` 中, 做一个备案, 记录我们浏览过的网页.

In [14]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import random


base_url = "https://baike.baidu.com"
his = ["/item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB/5162711"]

接着我们先不用循环, 对一个网页进行处理, 走一遍流程, 然后加上循环, 让我们的爬虫能在很多网页中爬取. 下面做的事情, 是为了在屏幕上打印出来我们现在正在哪张网页上, 网页的名字叫什么.

In [15]:
import lxml

url = base_url + his[-1]

html = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(html, features='lxml')
print(soup.find('h1').get_text(), '    url: ', his[-1])

网络爬虫     url:  /item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB/5162711


接下来我们开始在这个网页上找所有符合要求的 `/item/` 网址. 使用一个正则表达式过滤掉不想要的网址形式. 这样我们找到的网址都是 `/item/%xx%xx%xx...` 这样的格式了. 之后我们在这些过滤后的网页中随机选一个, 当做下一个要爬的网页. 不过有时候很不幸, 在 `sub_urls` 中并不能找到合适的网页, 我们就往回跳一个网页, 回到之前的网页中再随机抽一个网页做同样的事.

In [18]:
# find valid urls
sub_urls = soup.find_all("a", {"target": "_blank", "href": re.compile("/item/(%.{2})+$")})

if len(sub_urls) != 0:
    his.append(random.sample(sub_urls, 1)[0]['href'])
else:
    # no valid sub link found
    his.pop()
print(his)

['/item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB/5162711', '/item/%E4%B8%87%E7%BB%B4%E7%BD%91']


有了这套体系, 我们就能把它放在一个 `for loop` 中, 让它在各种不同的网页中跳来跳去.

In [20]:
his = ["/item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB/5162711"]

for i in range(20):
    url = base_url + his[-1]

    html = urlopen(url).read().decode('utf-8')
    soup = BeautifulSoup(html, features='lxml')
    print(i, soup.find('h1').get_text(), '    url: ', his[-1])

    # find valid urls
    sub_urls = soup.find_all("a", {"target": "_blank", "href": re.compile("/item/(%.{2})+$")})

    if len(sub_urls) != 0:
        his.append(random.sample(sub_urls, 1)[0]['href'])
    else:
        # no valid sub link found
        his.pop()

0 网络爬虫     url:  /item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB/5162711
1 深度优先策略     url:  /item/%E6%B7%B1%E5%BA%A6%E4%BC%98%E5%85%88%E7%AD%96%E7%95%A5
2 网络爬虫     url:  /item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB
3 网络数据     url:  /item/%E7%BD%91%E7%BB%9C%E6%95%B0%E6%8D%AE
4 网络爬虫     url:  /item/%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB
5 超文本     url:  /item/%E8%B6%85%E6%96%87%E6%9C%AC
6 文本     url:  /item/%E6%96%87%E6%9C%AC
7 正文     url:  /item/%E6%AD%A3%E6%96%87
8 文本     url:  /item/%E6%96%87%E6%9C%AC
9 正文     url:  /item/%E6%AD%A3%E6%96%87
10 文本     url:  /item/%E6%96%87%E6%9C%AC
11 解释学     url:  /item/%E8%A7%A3%E9%87%8A%E5%AD%A6
12 本体论     url:  /item/%E6%9C%AC%E4%BD%93%E8%AE%BA
13 胡塞尔     url:  /item/%E8%83%A1%E5%A1%9E%E5%B0%94
14 比利时     url:  /item/%E6%AF%94%E5%88%A9%E6%97%B6
15 凡尔登条约     url:  /item/%E5%87%A1%E5%B0%94%E7%99%BB%E6%9D%A1%E7%BA%A6
16 路易一世     url:  /item/%E8%B7%AF%E6%98%93%E4%B8%80%E4%B8%96
17 阿雷佐     url:  /item/%E9%98%BF%E9%9B%B7%E4%BD%90
18 皮耶罗·德拉·弗朗切斯卡     url:  /item/%E

这样我们就能观看我们的爬虫现在爬去了哪? 是不是爬到了和 “网页爬虫” 起始页完全不相关的地方去了.

接下来来的爬虫, 我们将要了解更多需要知道的功能. 用 requests 代替 urlopen, 还有如何从网页下载等.

---
## 6. 多功能的 Requests

### 6.1 获取网页的方式

其实在加载网页的时候, 有几种类型, 而这几种类型就是你打开网页的关键. 最重要的类型 (method) 就是 `get` 和 `post` (当然还有[其他的](https://www.w3schools.com/tags/ref_httpmethods.asp), 比如 `head`, `delete`). 刚接触网页构架的朋友可能又会觉得有点懵逼了. 这些请求的方式到底有什么不同? 他们又有什么作用?

我们就来说两个重要的, `get`, `post`, 95% 的时间, 你都是在使用这两个来请求一个网页.

* post
   * 账号登录
   * 搜索内容
   * 上传图片
   * 上传文件
   * 往服务器传数据 等
* get
   * 正常打开网页
   * **不**往服务器传数据

这样看来, 很多网页使用 `get` 就可以了, 而 `post`, 我们则是给服务器发送个性化请求, 比如将你的账号密码传给服务器, 让它给你返回一个含有你个人信息的 HTML.

从主动和被动的角度来说, `post` 中文是发送, 比较主动, 你控制了服务器返回的内容. 而 `get` 中文是取得, 是被动的, 你没有发送给服务器个性化的信息, 它不会根据你个性化的信息返回不一样的 HTML.

### 6.2 安装 requests

[Requests](http://docs.python-requests.org/en/master/) 是一个 Python 的外部模块, 我们需要手动安装它. 简单的方法, 在你的 terminal 或者是 cmd, 使用 pip 安装就好了.

    # python 3+
    pip3 install requests
    
[官网](http://docs.python-requests.org/en/master/user/install/#install)上还提供了其他途径的安装.

### 6.3 requests get 请求

有了 requests, 我们可以发送个中 method 的请求. 比如 `get`. 

首先我们需要观看一下百度搜索的规律. 在百度搜索框中写上 “莫烦python” 我们发现它弹出了一串这么长的网址.

![](https://morvanzhou.github.io/static/results/scraping/3-1-4.png)

但是仔细一看, 和 “莫烦Python” 有关的信息, 只有前面一小段 (“s?wd=莫烦python”), 其他的对我们来说都是无用的信息. 所以我们现在来尝试一下如果把后面的”无用” url 都去掉会怎样? Duang! 我们还是能搜到 “莫烦python”.

![](https://morvanzhou.github.io/static/results/scraping/3-1-5.png)

所以 “s?wd=莫烦python” 这就是我们搜索需要的关键信息. 我们就能用 `get` 来搭配一些自定义的搜索关键词来用 python 个性化搜索. 首先, 我们固定不动的网址部分是 “http://www.baidu.com/s”, `?` 后面的东西都是一些参数 (parameters), 所以我们将这些 parameters 用 python 的字典代替, 然后传入 requests.get() 功能. 然后我们还能用 python (webbrowser模块) 打开一个你的默认浏览器, 观看你是否在百度的搜索页面.

In [21]:
import requests
import webbrowser
param = {"wd": "莫烦Python"}  # 搜索的信息
r = requests.get('http://www.baidu.com/s', params=param)
print(r.url)
webbrowser.open(r.url)

http://www.baidu.com/s?wd=%E8%8E%AB%E7%83%A6Python


True

### 6.4 requests post 请求

`post` 又怎么用呢? 我们举个小例子, 在这个简单网页中, 我们有一个提交信息的窗口, 如果我提交上去这个信息, 那边的服务器会更加这个提交的信息返回出另一个网页. 这就是网页怎么样使用你 `post` 过去的信息了.

![](https://morvanzhou.github.io/static/results/scraping/3-1-6.png)

比如我在这里填上自己的姓名, 当我点 “submit” 的时候, 这个姓名(Morvan, Zhou) 就会被提交给服务器, 然后它会根据提交的姓名返回这个网页.

![](https://morvanzhou.github.io/static/results/scraping/3-1-7.png)

这样咋看起来好像和上面讲的 `get` 百度搜索没区别呀? 都是提交一些信息, 返回一个界面. 但是, **重点来了**. 你看看网址栏. 你 `post` 上去的个人信息, 有没有显示在 url 里? 你愿意将你的私密信息显示在 url 里吗? 你 `post` 过去的信息是交给服务器内部处理的. 不是用来显示在网址上的.

懂了这些, 我们就来看使用 python 和 requests 怎么做 `post` 这个操作吧.

首先我们调出浏览器的 `inspect` (右键点击 inspect, 中文是检查还是什么来着). 然后发现我们填入姓名的地方原来是在一个 `<form>` 里面.

![](https://morvanzhou.github.io/static/results/scraping/3-1-8.png)

这个 `<form>` 里面有一些 `<input>` 个 tag, 我们仔细看到 `<input>` 里面的这个值 `name="firstname"` 和 `name="lastname"`, 这两个就是我们要 `post` 提交上去的关键信息了. 我们填好姓名, 为了记录点击 “submit” 后, 浏览器究竟发生了什么翻天覆地的变化, 我们在 `inspect` 窗口, 选择 `Network`, 勾选 `Preserve log`, 再点击 “submit”, 你就能看到服务器返回给你定制化后的页面时, 你使用的方法和数据.

![](https://morvanzhou.github.io/static/results/scraping/3-1-9.png)

这些数据包括了:

* Request URL (post 要用的 URL)
* Request Method (post)
* Form Data (post 去的信息)

有了这些记录, 我们就能开始写 Python 来模拟这一次提交 post 了. 根据 `'firstname'` 和 `'lastname'`, 也就是上图里面的 Form data, 组织成一个 python 字典. 让后把这个字典传入 `requests.post()`, 注意, 这里的 post 里面的 url, 不是我们填表时的 url (`http://pythonscraping.com/pages/files/form.html`), 而是要把 Form 信息提交去的那个网页, 也就是上图中查看到的 Request URL (`http://pythonscraping.com/files/processing.php`).

In [23]:
data = {'firstname': '沛', 'lastname': '苗'}
r = requests.post('http://pythonscraping.com/files/processing.php', data=data)
print(r.text)

Hello there,  !


通过这个练习, 我们对 HTML 中的 Form 有了理解, 学会了怎么样使用 python 来提交 Form, 登录上提交后的页面.

### 6.5 上传图片

传照片也是 `post` 的一种, 我们得将本地的照片文件传送到服务器. 我们使用这个[网页](http://pythonscraping.com/files/form2.html)来模拟一次传照片的过程.

![](https://morvanzhou.github.io/static/results/scraping/3-1-10.png)

如果你留意观察 url, 你会发现, 传送完照片以后的 url 有变动. 我们使用同样的步骤再次检查, 发现, “choose file” 按键链接的 `<input>` 是一个叫 `uploadFile` 的名字. 我们将这个名字记下, 放入 python 的字典当一个 “key”.

![](https://morvanzhou.github.io/static/results/scraping/3-1-11.png)

接着在字典中, 使用 open 打开一个图片文件, 当做要上传的文件. 把这个字典放入你的 `post` 里面的 `files` 参数. 就能上传你的图片了, 网页会返回一个页面, 将你的图片名显示在上面.

In [25]:
file = {'uploadFile': open('data/mindmap.png', 'rb')}
r = requests.post('http://pythonscraping.com/files/processing2.php', files=file)
print(r.text)

uploads/
Sorry, there was an error uploading your file.


### 6.6 登录

用 `post` 还有一个重要的, 就是模拟登录. 再登录的时候发生了什么事情呢? 我们使用这个[简单的登录网页](http://pythonscraping.com/pages/cookies/login.html)进行说明.

![](https://morvanzhou.github.io/static/results/scraping/3-1-12.png)

通过之前提到的方法, 我们观察一下浏览器给出的记录. 三个重要的方面都被我圈出来了.

![](https://morvanzhou.github.io/static/results/scraping/3-1-13.png)

我们总结一下, 为了这次登录账号, 我们的浏览器做了什么.

1. 使用 post 方法登录了第一个红框的 url
2. post 的时候, 使用了 Form data 中的用户名和密码
3. **生成了一些 cookies**

第三点我们是从来没有提到过的. cookie, 听起来很熟呀! 每当游览器出现问题的时候, 网上的解决方法是不是都有什么清除 cookie 之类的, 那 cookie 实际上是什么呢? [这里](https://baike.baidu.com/item/cookie/1119?fr=aladdin)给出了和全面的介绍.

简单来说, 因为打开网页时, 每一个页面都是不连续的, 没有关联的, cookies 就是用来衔接一个页面和另一个页面的关系. 比如说当我登录以后, 浏览器为了保存我的登录信息, 将这些信息存放在了 cookie 中. 然后我访问第二个页面的时候, 保存的 cookie 被调用, 服务器知道我之前做了什么, 浏览了些什么. 像你在网上看到的广告, 为什么都可能是你感兴趣的商品? 你登录淘宝, 给你推荐的为什么都和你买过的类似? 都是 cookies 的功劳, 让服务器知道你的个性化需求.

所以大部分时候, 每次你登录, 你就会有一个 cookies, 里面会提到你已经是登录状态了. 所以 cookie 在这时候很重要. cookies 的传递也特别重要, 比如我用 `requests.post + payload` 的用户信息发给网页, 返回的 `r` 里面会有生成的 cookies 信息. 接着我请求去登录后的页面时, 使用 `request.get`, 并将之前的 cookies 传入到 get 请求. 这样就能已登录的名义访问 get 的页面了.

In [28]:
payload = {'username': 'Morvan', 'password': 'password'}
r = requests.post('http://pythonscraping.com/pages/cookies/welcome.php', data=payload)
print(r.cookies.get_dict())

# {'username': 'Morvan', 'loggedin': '1'}

r = requests.get('http://pythonscraping.com/pages/cookies/profile.php', cookies=r.cookies)
#print(r.text)

{'loggedin': '1', 'username': 'Morvan'}


![](https://morvanzhou.github.io/static/results/scraping/3-1-14.png)

### 6.7 使用 Session 登录

不过每次都要传递 cookies 是很麻烦的, 好在 requests 有个很 handy 的功能, 那就是 `Session`. 在一次会话中, 我们的 cookies 信息都是相连通的, 它自动帮我们传递这些 cookies 信息. 这时我感叹, 程序员真会偷懒~ 哈哈.

同样是执行上面的登录操作, 下面就是使用 session 的版本. 创建完一个 session 过后, 我们直接只用 session 来 `post` 和 `get`. 而且这次 `get` 的时候, 我们并没有传入 cookies. 但是实际上 session 内部就已经有了之前的 cookies 了.

In [29]:
session = requests.Session()
payload = {'username': 'Morvan', 'password': 'password'}
r = session.post('http://pythonscraping.com/pages/cookies/welcome.php', data=payload)
print(r.cookies.get_dict())

# {'username': 'Morvan', 'loggedin': '1'}

r = session.get("http://pythonscraping.com/pages/cookies/profile.php")
#print(r.text)

{'loggedin': '1', 'username': 'Morvan'}


---
## 7. 下载文件

### 7.1 下载之前

在下载之前, 我们的弄清楚怎么样下载. 打个比方, 以前有朋友留言说我的教程太多, 不知道从何学起, 我专门制作了一张学习流程图, 这张[莫烦Python的个性化学习路线](https://morvanzhou.github.io/learning-steps/)应该也拯救了无数迷途少年吧. 哈哈哈, 今天我们来爬这张图, 还有下载这张图.

![](https://morvanzhou.github.io/static/img/description/learning_step_flowchart.png)



## Reference

[莫烦 python爬虫](https://morvanzhou.github.io/tutorials/data-manipulation/scraping/1-00-why/)

[Python2爬虫学习系列教程](https://cuiqingcai.com/1052.html)

[零基础如何学爬虫技术？](https://www.zhihu.com/question/47883186)

[一看就明白的爬虫入门讲解：基础理论篇](https://www.csdn.net/article/2015-11-13/2826205)

[]()

[]()

[]()

[]()

[]()

[]()

[]()

[]()

[]()

[]()

[]()

[]()